# Imports

In [1]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# import urllib.parse

# # Setup Chrome options
# options = Options()
# # options.add_argument("--start-maximized")  # Optional: Maximize window
# # options.add_argument("--headless")  # Optional: Run headless if GUI is not needed

# # Install & initialize WebDriver automatically|
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)

# # Set implicit wait
# driver.implicitly_wait(10)

print("WebDriver initialized successfully!")

import requests
import pandas as pd
from bs4 import BeautifulSoup

# Define the headers as seen in the request
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9,bn;q=0.8",
    "Origin": "https://map.gulfood.com",
    "Referer": "https://map.gulfood.com/",
    "sec-ch-ua": '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
    "x-map-id": "obl0gRq3GXBm1"
}

WebDriver initialized successfully!


# 1st Step -> Get Counties List

In [ ]:
records = []
for i in range(0, 52):
    driver.get('https://data.bls.gov/multi-screen?survey=la')
    time.sleep(1)

    all_states_elm = driver.find_elements(By.XPATH, '//select[@id="selectbox-single"]/option')

    ## Select State/Region
    state_selected = all_states_elm[i].text
    print(f"{state_selected}")
    all_states_elm[i].click()

    ## Click - Next form
    driver.find_element(By.XPATH, '//button[@id="sa-next-button"]').click()
    time.sleep(1)

    ## Select [OPTION]- Counties and equivalents
    options = driver.find_elements(By.XPATH, '//select[@id="selectbox-single"]/option')

    # Loop through the options and select the one that matches
    found = False
    for option in options:
        if "F Counties and equivalents" in option.text:
            option.click()
            found = True
            time.sleep(1)
            break
    
    if found == False:
        print(f'\n[CAUTION]: Counties and equivalents option not found.\n')
        continue
    ## Click - Next form
    driver.find_element(By.XPATH, '//button[@id="sa-next-button"]').click()
    time.sleep(1)

    county_elements = driver.find_elements(By.XPATH, '//select[@class="form-select ng-untouched ng-pristine ng-invalid"]/option')
    for el in county_elements:
        info = [state_selected, el.text]
        if info not in records:
            records.append(info)

    print(f'Total Counties Found on {state_selected}: {len(county_elements)} | Records: {len(records)}')

    # break

## Saving the DataFrame
df = pd.DataFrame(records, columns=['State', 'County'])

# Step 1: Split into Code + Rest
df[['County_Code', 'Rest']] = df['County'].str.split(' ', n=1, expand=True)

# Step 2: Split Rest into Name + State (renaming extracted State to avoid overwrite)
df[['County_Name', 'County_State']] = df['Rest'].str.rsplit(',', n=1, expand=True)

# Step 3: Clean whitespace
df['County_Name'] = df['County_Name'].str.strip()
df['County_State'] = df['County_State'].str.strip()

# Drop the helper column
df = df.drop(columns=['Rest'])
df.to_excel('bls_counties_list.xlsx', index=False)

# Second Step

### Functions In Use

In [3]:
import requests
from io import StringIO
from IPython.display import clear_output


def save_output(dfs, prefix = 'final'):
    wf = pd.concat(dfs)
    wf.to_pickle(f'output/{prefix}_bls_counties_data_sample.pkl')

    return wf

def fetch_bls_series(series_id: str):
    """
    Fetch BLS data for a given series_id from SurveyOutputServlet.
    
    Args:
        series_id (str): The BLS series ID (e.g., 'LAUCN010010000000003').
    
    Returns:
        dict: JSON response from the API
    """
    url = "https://data.bls.gov/pdq/SurveyOutputServlet"

    payload = {
        "request_action": "get_data",
        "reformat": "true",
        "from_results_page": "true",
        "from_year": "1990",
        "to_year": "2025",
        "Go.x": "9",
        "Go.y": "3",
        "initial_request": "false",
        "data_tool": "dsrv",
        "series_id": series_id,
        "output_type": "simple",
        "original_output_type": "simple",
        "original_annualAveragesRequested": "false"
    }

    response = requests.post(url, data=payload)
    response.raise_for_status()
    return response

def parse_catalog_info(rows):
    """
    Parse catalog information from <tr> elements into structured variables.
    
    Args:
        rows (list): List of BeautifulSoup <tr> elements.
    
    Returns:
        dict: Dictionary with structured variables.
    """
    # Initialize variables as None (so we know if something is missing)
    series_id = None
    area = None
    area_type = None
    state_region_division = None
    measure = None
    seasonal = None

    for row in rows:
        th = row.find("th")
        td = row.find("td")

        if th and td:
            key = th.get_text(strip=True).replace(":", "")
            value = td.get_text(strip=True)

            if key == "Series Id":
                series_id = value
            elif key == "Area":
                area = value
            elif key == "Area Type":
                area_type = value
            elif key == "State/Region/Division":
                state_region_division = value
            elif key == "Measure":
                measure = value
        elif td and not th:
            # Handles rows like "Not Seasonally Adjusted"
            seasonal = td.get_text(strip=True)

    return {
        "Series_Id": series_id,
        "Seasonal": seasonal,
        "Area": area,
        "Area_Type": area_type,
        "State_Region_Division": state_region_division,
        "Measure": measure
    }

def get_table_df(response):
    soup = BeautifulSoup(response.text, 'lxml')
    basic_info = parse_catalog_info(soup.select('#catalog0 tr'))

    # Extract the HTML table as a string
    table_html = StringIO(str(soup.select_one('#table0')))

    # Parse with pandas
    sf = pd.read_html(table_html)[0]

    # Remove the last row
    sf = sf.iloc[:-1]


    ## Final Work
    sf['series_id'] = basic_info['Series_Id']
    sf['Area'] = basic_info['Area']
    sf['Area_Type'] = basic_info['Area_Type']
    sf['State_Region_Division'] = basic_info['State_Region_Division']
    sf['Measure'] = basic_info['Measure']

    sf = sf[['series_id', 'Area', 'Area_Type', 'State_Region_Division', 'Measure', 'Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

    return sf

def convert_county_code(county_code: str) -> str:
    """
    Convert a county code like 'CN4202500000000' into 
    a BLS series ID like 'LAUCN420250000000003'.

    Args:
        county_code (str): County code starting with 'CN'

    Returns:
        str: Converted BLS series ID
    """
    return f"LAU{county_code}03"


In [23]:
df[df['County_Code'].str.len() == 1]


,State,County,County_Code,County_Name,County_State,dm_code
2782,49 Utah,A Statewide,A,Statewide,,LAUA03
2783,49 Utah,B Metropolitan areas,B,Metropolitan areas,,LAUB03
2784,49 Utah,D Micropolitan areas,D,Micropolitan areas,,LAUD03
2785,49 Utah,E Combined areas,E,Combined areas,,LAUE03
2786,49 Utah,F Counties and equivalents,F,Counties and equivalents,,LAUF03
2787,49 Utah,"G Cities and towns above 25,000 population",G,Cities and towns above 25,000 population,LAUG03
2788,49 Utah,I Parts of cities that cross county boundaries,I,Parts of cities that cross county boundaries,,LAUI03
2789,49 Utah,J Multi-entity small labor market areas,J,Multi-entity small labor market areas,,LAUJ03
2790,49 Utah,K Intrastate parts of interstate areas,K,Intrastate parts of interstate areas,,LAUK03


True

In [16]:
soup = BeautifulSoup(response.text, 'lxml')
basic_info = parse_catalog_info(soup.select('#catalog0 tr'))

In [4]:
df = pd.read_excel('bls_counties_list.xlsx')
df = df.fillna('')
df['dm_code'] = df['County_Code'].apply(convert_county_code)

In [26]:
# dfs = []

for i in range(i, len(df)):
    code = df.at[i, 'dm_code']
    
    if len(code) == 6:
        continue
    
    response = fetch_bls_series(code)
    sf = get_table_df(response)
    dfs.append(sf)
    print(f'{i}: {code}')

    if i % 50 == 0:
        wf = save_output(dfs, prefix=f'backup')
        clear_output(wait=True)
        print(f'{i}: {code} - Backup saved.')

wf = save_output(dfs, prefix=f'Final')

3200: LAUCN721450000000003 - Backup saved.
3201: LAUCN721470000000003
3202: LAUCN721490000000003
3203: LAUCN721510000000003
3204: LAUCN721530000000003


In [1]:
import pandas as pd

In [ ]:
# df = pd.read_pickle('output/Final_bls_counties_data_sample.pkl')
# df.to_excel('MASTER_SHEET.xlsx', index = False)

In [8]:
len(df['series_id'].unique())

3196